In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import openpyxl



staryear = input("Ingresar un año de inicio de la estadística (1947-2016): ")
endyear = input("Ingresar un año de fin de la estadística (1947-2016): ")
for i in range(staryear, endyear+1):
    
if int(mes) >9:
    url = f'https://wdc.kugi.kyoto-u.ac.jp/dst_final/{año}{mes}/index.html'
else:
    url = f'https://wdc.kugi.kyoto-u.ac.jp/dst_final/{año}0{mes}/index.html'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# Obtención de datos del texto
data_text = soup.get_text()

# Encontrar números enteros en el texto
data_values = [int(value) for value in re.findall(r'-?\d+', data_text)]

# Dividimos los datos en sublistas para cada día
day_length = 24
all_day_data = [data_values[i:i+day_length] for i in range(26, len(data_values), day_length+1)]

dataframe = pd.DataFrame(all_day_data, index=range(1,len(all_day_data)+1), columns=range(1,len(all_day_data[0])+1))
print(dataframe)

# Nombre para el archivo exportado
nombre = 'datos('+año+'-'+mes+').xlsx'

# Exportar dataframe a excel
dataframe.to_excel('exports/'+nombre, engine='openpyxl',sheet_name='IndiceDST')

# Abrir archivo exportado
dataframe = pd.read_excel('exports/'+nombre)

#Funcion para extraer dstIndex diario
def tipoTormenta(all_day_data):
    vector = []
    vectorVerificar = []

    for i in range(len(all_day_data)):
        for j in range(len(all_day_data[i])):

            if all_day_data[i][j] < -100:
                vectorVerificar.append(2)

            elif all_day_data[i][j] > -100 and all_day_data[i][j] < -50:
                vectorVerificar.append(1)
                
            else:
                vectorVerificar.append(0)

        if 2 in vectorVerificar:
            vector.append("Intensa")

        elif 1 in vectorVerificar:
            vector.append("Moderada")
        else:
            vector.append("Debil")

        vectorVerificar.clear()


    print("---")
    print(vectorVerificar)
    print(vector)
    print("Tamaño vector: ", len(vector))

 

    tipoTormentaDataFrame = pd.DataFrame(vector)
    tipoTormentaDataFrame.index += 1
    tipoTormentaDataFrame.columns += 1
    with pd.ExcelWriter(f'exports/{nombre}', engine='openpyxl', mode='a') as writer:
        indice = len(vector)
        tipoTormentaDataFrame.to_excel(writer, sheet_name='Tipo_Tormenta')
        
    print(tipoTormentaDataFrame)

tipoTormenta(all_day_data)




    1   2   3   4   5   6   7   8   9   10  ...  15  16  17  18  19  20  21  \
1    3   5  10  14  -1  -9 -14 -20 -27 -33  ... -41 -43 -39 -35 -31 -27 -27   
2  -27 -29 -29 -27 -32 -31 -32 -27 -26 -22  ... -16 -14  -9  -5  -5  -7  -6   
3   -3  -3  -7 -10 -10 -10  -9  -7  -9  -9  ...  -4  -2   3   2  -5 -10 -11   
4   -3  -1  -3  -3  -4  -4  -3  -3  -2  -1  ...   1  -3  -7  -5   0  -2   1   
5   12  15  15  11   2  -2  -2   0   1   0  ...   2   1  -1  -2   0  -4 -10   
6   -7   1   7   0  -5  -2  -5  -6 -11 -15  ...  -2   1  -4  -9 -11 -15 -22   
7  -26 -29 -27 -28 -37 -36 -29 -33 -37 -35  ... -19 -21 -22 -20 -17 -20 -27   
8  -25 -28 -23 -19 -21 -22 -21 -20 -23 -26  ... -24 -25 -32 -31 -28 -34 -43   
9  -35 -28 -21 -17 -16 -18 -19 -19 -18 -17  ... -10  -9  -8  -7  -6  -5  -5   
10  -2   1   2   2   4   4   4   2  -3 -10  ... -18 -21 -21 -26 -30 -33 -35   
11 -40 -41 -39 -41 -41 -37 -35 -39 -38 -33  ... -33 -31 -28 -31 -36 -37 -40   
12 -38 -34 -31 -27 -19 -27 -32 -22 -19 -22  ... -11 

OSError: Cannot save file into a non-existent directory: 'exports'